In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [2]:
!wget -c https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py


--2018-12-28 07:45:18--  https://raw.githubusercontent.com/udacity/deep-learning-v2-pytorch/master/intro-to-pytorch/helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 416 Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [3]:
#downloading dataset
!wget -c https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
!unzip -qq Cat_Dog_data.zip

--2018-12-28 07:45:22--  https://s3.amazonaws.com/content.udacity-data.com/nd089/Cat_Dog_data.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.165.69
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.165.69|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

replace Cat_Dog_data/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Cat_Dog_data/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Cat_Dog_data/test/cat/cat.7660.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/Cat_Dog_data/test/cat/._cat.7660.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace Cat_Dog_data/test/cat/cat.1211.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [5]:
!pip install Pillow==5.3.0


In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets , transforms , models

data_dir = 'Cat_Dog_data/train'

train_transforms = transforms.Compose([transforms.RandomRotation(30),transforms.RandomResizedCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(255),transforms.CenterCrop(224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

train_data = datasets.ImageFolder(data_dir , transform=train_transforms)
test_data = datasets.ImageFolder(data_dir , transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=64)

In [20]:
#here we are using resnet and so it has two parts "features" and "fc" whereas we had features and classifer
## Use GPU if it's available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#using pre-trained resnet50 from imageNet database
model = models.resnet50(pretrained=True)

## Freeze parameters so we don't backprop through them

for param in model.parameters():
    param.requires_grad = False

#creating our own classifier with name fc (as defined in resnet50)
model.fc = nn.Sequential(nn.Linear(2048, 512),nn.ReLU(),nn.Dropout(0.2),nn.Linear(512, 2),nn.LogSoftmax(dim=1))    

#using crossentropyloss
criterion = nn.NLLLoss()

#taking steps of gradient descent with lr =0.003 and only fc parameters ,Only train the classifier parameters, feature parameters are frozen

optimizer = optim.Adam(model.fc.parameters(), lr=0.003)

model.to(device);


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:01<00:00, 85947875.13it/s]


In [21]:
#printing after every 5 steps , so no of print = batchsize/5
epochs = 1
steps = 0
running_loss = 0
print_every = 5
for epoch in range(epochs):
    for inputs, labels in trainloader:
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        #backprop for classifier
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        if steps % print_every == 0: #only true after every 5 steps , like 5,10,15 .... done to check overfitting
            test_loss = 0
            accuracy = 0
            model.eval()  #swicthing off dropout as it is only used for training and not for actual inference or validation acuracy measurements
            # turn off gradients
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    batch_loss = criterion(logps, labels)
                    
                    test_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            print(f"Epoch {epoch+1}/{epochs}.. "
                  f"Train loss: {running_loss/print_every:.3f}.. "
                  f"Test loss: {test_loss/len(testloader):.3f}.. "
                  f"Test accuracy: {accuracy/len(testloader):.3f}")
            running_loss = 0
            model.train()        

Epoch 1/1.. Train loss: 2.214.. Test loss: 2.104.. Test accuracy: 0.500
Epoch 1/1.. Train loss: 1.255.. Test loss: 0.937.. Test accuracy: 0.644
Epoch 1/1.. Train loss: 0.698.. Test loss: 0.438.. Test accuracy: 0.809
Epoch 1/1.. Train loss: 0.373.. Test loss: 0.128.. Test accuracy: 0.950
Epoch 1/1.. Train loss: 0.217.. Test loss: 0.090.. Test accuracy: 0.968
Epoch 1/1.. Train loss: 0.199.. Test loss: 0.077.. Test accuracy: 0.975
Epoch 1/1.. Train loss: 0.190.. Test loss: 0.090.. Test accuracy: 0.965


KeyboardInterrupt: ignored